In [1]:
import numpy as np
from numba import njit
from numba import types
from numba.typed import Dict

In [2]:
number = [120,250,500,1000]

# アルゴリズム

In [3]:
def E_dict_func(E): #キーにアイテム名、値にそのアイテムとコンフリクトのあるアイテムをもつ辞書
    E_dict={}
    for e in E:
        E_dict[e[0]] = tuple(set(E_dict.setdefault(e[0],()) + (e[1],)))
        E_dict[e[1]] = tuple(set(E_dict.setdefault(e[1],()) + (e[0],)))
    return E_dict

In [4]:
def nonconf_E_dict_func(E,I): #キーにアイテム名、値にそのアイテムとコンフリクトのないアイテムをもつ辞書
    if len(E)==0:
        return {}
    else:
        E_dict = {}
        nonconf_E_dict={}
        E_dict = E_dict_func(E)
        nonconf_E_dict = {i:tuple(set(I)-(set(E_dict[i])|{i})) if i in E_dict.keys() else tuple(set(I)-{i}) for i in I}

        return nonconf_E_dict

In [5]:
def nonconf_E_func(E,I): #枝集合　アイテムiとjにコンフリクトがないとき、枝(i,j)をつくる
    nonconf_E = []
    nonconf_E_dict = nonconf_E_dict_func(E,I)
    for i in nonconf_E_dict:
        for j in nonconf_E_dict[i]:
            if i != j and (j,i) not in nonconf_E:
                nonconf_E.append((i,j))
            
    return nonconf_E

In [6]:
def update_E(E,I): #アイテムの分割の際、コンフリクトがあるアイテム間の枝の集合を更新する
    return [e for e in E if ((e[0] in I) or (e[1] in I)) and ((e[1],e[0]) not in E)]

In [7]:
def update_nonconf_E(E,I1,I2):
    I1_nonconf_E = []
    I2_nonconf_E = []
    
    I1_nonconf_E = nonconf_E_func(E,I1)
    I2_nonconf_E = nonconf_E_func(E,I2)
    return I1_nonconf_E,I2_nonconf_E

In [8]:
def update_M(M,I):
    return list(set(tuple(set(I)&set(g)) for g in M if len(set(I)&set(g))!=0))

In [34]:
#@njit
def capacity(w_i,bin_b,w,C): #bin_bの容量がw_iの大きさよりも大きいかを判定
    if (C-sum(w[i] for i in bin_b)) >= w_i:
        return True
    else:
        return False

In [10]:
def conflict(item,item_in_bin_b,E_dict): #bin_bに割り当てられているアイテムたちとw_iの間にコンフリクトがあるかを判定
    if item not in E_dict.keys():
        return True
    for i in item_in_bin_b:
        if i in E_dict[item]:
                return False
    return True

In [11]:
def group(i,M): #アイテムiの属するグループの番号を返す関数
    for index,menber in enumerate(M):
        if i in menber:
            return index

In [12]:
def degree(i,E_dict): #アイテムiのコンフリクトの次数を返す関数
    try:
        return len(E_dict[i])
    except:
        return 0

In [13]:
def sort_M(M,E_dict,r):
    if M!=[] and E_dict!={}:
        return sorted(M,key=lambda x:(r[M.index(x)], max(len(E_dict[i]) if i in E_dict.keys() else 0 for i in x)),reverse=True)
    else:
        return []

# H1

In [32]:
def FFD(I,C,w,E,M,assign): #アイテムをビンに割り当てる
    E_dict = {}
    E_dict = E_dict_func(E) #コンフリクトを表す枝を管理する辞書
    for i in I:
        for b in range(len(assign.keys())+1): #インデックスが小さい順にビンを走査
            assign[b] = assign.setdefault(b,[])
            if capacity(w[i],assign[b],w,C) or assign[b]==[]: #容量制約を満たす
                assign[b].append(i) #ビンbにアイテムiを入れる
                break  
                
    return assign

# H2

In [31]:
def FFD_for_conflict(I,C,w,E,M,assign): #コンフリクトの数が多い順にアイテムをビンに割り当てる
    E_dict = {}
    E_dict = E_dict_func(E) #コンフリクトを表す枝を管理する辞書
    for i in I:
        for b in range(len(assign.keys())+1): #インデックスが小さい順にビンを走査
            assign[b] = assign.setdefault(b,[])
            if (capacity(w[i],assign[b],w,C) and conflict(i,assign[b],E_dict)) or assign[b]==[]: #容量制約を満たす　and　コンフリクトがない
                assign[b].append(i) #ビンbにアイテムiを入れる
                break  

    return assign

# H3

In [16]:
def H3(I,C,w,E,M,r,assign): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    E_dict = {}
    M_prime = []
    
    E_dict = E_dict_func(E) #コンフリクトを表す枝を管理する辞書
    M_prime = update_M(M,I) #グループの順序つき集合Mを更新する。
    
    M_prime = sort_M(M_prime,E_dict,r)
    
    #グループの集合Mはすでに並べ替えてある
    for g in M_prime: #グループごとにFFD_for_conflict()
        assign = FFD_for_conflict(g,C,w,E,M_prime,assign) #gに対してFFD for conflictを行う
        
    return assign

# H4

In [17]:
def H4(I,C,w,E,M,r,assign): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    E_dict = {}
    M_prime = []
    
    E_dict = E_dict_func(E) #コンフリクトを表す枝を管理する辞書
    
    H = list(set(I)-set(E_dict.keys())) #アイテムの順序付き集合I内のコンフリクトの数が0のアイテムの集合をHとする
    I = set(I)-set(H) #I=I\Hとする。
    I = sorted(list(I),key=lambda x:w[x],reverse=True) #アイテム集合をアイテムの大きさが大きい順に並べ替える
    
    M_prime = update_M(M,I)#[tuple(set(I)&set(g)) for g in M if len(set(I)&set(g))!=0] #グループの順序つき集合Mを更新する。
    
    M_prime = sort_M(M_prime,E_dict,r)

    for g in M_prime: #グループごとにFFD_for_conflict()
        assign = FFD_for_conflict(g,C,w,E,M_prime,assign) #gに対してFFD for conflictを行う
            
    assign = FFD(H,C,w,E,M_prime,assign) #Hに対してFFDを行う
        
    return assign

# H5

In [18]:
def H5(I,C,w,E,M,r,assign): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    E_dict = {}
    M_prime = []
    
    E_dict = E_dict_func(E) #コンフリクトを表す枝を管理する辞書
    M_prime = update_M(M,I)#[tuple(set(I)&set(g)) for g in M if len(set(I)&set(g))!=0] #グループの順序つき集合Mを更新する。
    
    M_prime = sort_M(M_prime,E_dict,r)

    #グループの集合Mはすでに並べ替えてある
    for g in M_prime: #グループごとにFFD_for_conflict()
        H = list(set(g)-set(E_dict.keys())) #アイテムの順序付き集合I内のコンフリクトの数が0のアイテムの集合をHとし、I=I\Hとする。
        g = set(g)-set(H) #g=g\H
        g = sorted(list(g),key=lambda x:w[x],reverse=True) #アイテム集合をアイテムの大きさが大きい順に並べ替える
        #print("H\t",H)
        #print("g\t",g)
        assign = FFD_for_conflict(g,C,w,E,M_prime,assign) #gに対してFFD for conflictを行う
        assign = FFD(H,C,w,E,M_prime,assign) #Hに対してFFDを行う
        
    return assign

# H6

In [19]:
def H6(I,C,w,E,M,r,assign): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    E_dict = {}
    noncconf_E_dict = {}
    M_prime = []
    
    E_dict = E_dict_func(E) #コンフリクトを表す枝を管理する辞書
    nonconf_E_dict = nonconf_E_dict_func(E,I)
    M_prime = update_M(M,I)#[tuple(set(I)&set(g)) for g in M if len(set(I)&set(g))!=0] #グループの順序つき集合Mを更新する。
    
    M_prime = sort_M(M_prime,E_dict,r)
    
    #グループの集合Mはすでに並べ替えてある
    for g in M_prime:
        H = list(set(g)-set(E_dict.keys())) #アイテムの順序付き集合I内のコンフリクトの数が0のアイテムの集合をHとし、I=I\Hとする。
        g = tuple(set(g)-set(H)) #g=g\H
        #print("H\t",H)
        #print("g\t",g)
        while len(g)>0:
            max_degree = max(g,key=lambda x:len(E_dict[x])) #グループg内で最もコンフリクトの数が多いアイテムiを選ぶ
            #print("max_degree\t",max_degree)
            if len(g)!=1:
                g_i = (max_degree,) + tuple(menber for menber in g if menber in nonconf_E_dict[max_degree]) #グループg内でiとコンフリクトがないアイテムとiからなる集合g_iを定める
            else:
                g_i = (max_degree,)
            #print("g_i\t",g_i)
            assign = FFD_for_conflict(g_i,C,w,E,M_prime,assign) #g_iに対してFFD for conflictを行う
            g = tuple(set(g)-set(g_i)) #g=g\g_i
                
        assign = FFD(H,C,w,E,M_prime,assign) #Hに対してFFDを行う

    return assign

# H7

In [20]:
def H7(I,C,w,E,M,r,assign): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    E_dict = {}
    M_prime = []
    
    #E_dict = E_dict_func(E) #コンフリクトを表す枝を管理する辞書
    H = list(set(I)-set(E_dict.keys())) #アイテムの順序付き集合I内のコンフリクトの数が0のアイテムの集合をHとし、I=I\Hとする。
    I = set(I)-set(H)
    I = sorted(list(I),key=lambda x:w[x],reverse=True) #アイテム集合をアイテムの大きさが大きい順に並べ替える
    
    M_prime = [tuple(set(I)&set(g)) for g in M if len(set(I)&set(g))!=0] #グループの順序つき集合Mを更新する。
    
    #E = update_E(E,I)
    E_dict = E_dict_func(E) #コンフリクトを表す枝を管理する辞書
    nonconf_E_dict = nonconf_E_dict_func(E,I) #コンフリクトを表す枝を管理する辞書
    
    M_prime = sort_M(M_prime,E_dict,r)
 
    for g in M_prime:
        while len(g)>0:
            #コンフリクトが最高次数のアイテムを選べてない
            max_degree = max(g,key=lambda x:len(E_dict[x])) #グループg内で最もコンフリクトの数が多いアイテムiを選ぶ
            if len(g)!=1:
                #すでに割り当てたアイテムをg_iに含んでる可能性がある気がする
                g_i = (max_degree,) + tuple(menber for menber in g if menber in nonconf_E_dict[max_degree]) #グループg内でiとコンフリクトがないアイテムとiからなる集合g_iを定める
            else:
                g_i = (max_degree,)
            assign = FFD_for_conflict(g_i,C,w,E,M_prime,assign) #g_iに対してFFD for conflictを行う
            g = tuple(set(g)-set(g_i)) #g=g\g_i
                
    assign = FFD(H,C,w,E,M_prime,assign) #Hに対してFFDを行う

    return assign

In [21]:
def clique(SUB,A):
    flag=True
    for i in SUB:
        for j in SUB:
            if i!=j:
                if (i,j) not in A and (j,i) not in A:
                    flag = False
                    break
        if flag!=True:
            break       
    return flag

In [22]:
def fewest_degree_node(SUB,A):
    degree_dict = {}
    for n in SUB:
        degree_dict[n] = 0
    for e in A:
        try:
            degree_dict[e[0]] += 1
            degree_dict[e[1]] += 1
        except:
            continue
    return {min(degree_dict,key=lambda x:degree_dict[x])}

In [23]:
def Johnson(N,A): #与えられた点集合と枝集合から最大クリークを生成する #Nは点の集合、Aは枝の集合
    SUB = N
    while True:
        if clique(SUB,A):
            break
        else:
            y = fewest_degree_node(SUB,A) #SUBのノードのうち、最も次数の少ないものを定める
            SUB = SUB-y #SUBからyを除く
            for e in A:
                if y in e:
                    A = A-{e} #yにつながってる枝を除く
    return SUB

In [24]:
def Gendreau(I,C,w,E,M,assign):
    #print("Gendreau start")
    
    import copy
    
    E_dict = E_dict_func(E) #コンフリクトを表す枝を管理する辞書
    nonconf_E = nonconf_E_func(E,I)

    #1
    V = set(I)
    H = set(I)-set(E_dict.keys()) #アイテムの集合V内のコンフリクトの数が0のアイテムの集合をHとし、V=V\Hとする。
    V = V-H #コンフリクトの数が0のアイテムを含まないアイテム集合
    E = set(E)
    
    #print("V\t",V)
    #print("H\t",H)

    #2
    assign_temp = {}
    while V!=set():
        if V==set(): #V=φならば終了
            break
        else:
            D = Johnson(V,E) #DはグラフG内の最大のクリーク
            V = V-D

        #3
        for i in D:
            V = V | {i}
            D_i = Johnson(V,nonconf_E) | {i} #V ∪ {i} と　コンフリクトがないアイテム同士の組みを要素にもつ枝集合　から最大クリークを見つける
            #print("i\t",i)
            #print("D_i\t",D_i)
            #print("E\t",E)
            
            assign_temp = FFD_for_conflict(D_i,C,w,E,M,assign_temp)
            #print("assign_temp\t",assign_temp)
            
            for index,value in assign_temp.items():
                if i in value:
                    V = V - set(value)
                    for new_key in range(len(assign_temp.keys())):
                        #print("新しくビンに割り当てられたアイテム\t",set(value)&D_i )
                        #print("new_key\t",new_key)
                        
                        if new_key not in assign.keys(): #new_keyがassignに登録されていない場合
                            #print("新しいビンを使う")
                            assign[new_key] = value
                            break
                        
                        elif set(assign[new_key])|(set(value)&D_i)==set(value):#assign_temp[index]: #new_keyはassignに登録されていて、正しい割り当てビンの場合
                            #print("今あるビンに入れる")
                            #print(f"ビン{index}にアイテム{set(value)&D_i }を入れる")
                            assign[new_key] = list(set(assign[new_key]) | set(value))
                            break
                                
                    #print("assign\t",assign)
                        
            assign_temp = copy.deepcopy(assign)
            D = D - {i}

            B = {}
            for b in assign.keys():
                B[b] = C-sum(w[i] for i in assign[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
            #print("B\t",B)
            #print()

    #4
    assign = FFD(H,C,w,E,M,assign) #Hに対してFFDを行う
    #print("HにFFD")
    #print("H\t",H)
    #print("assign\t",assign)
    #print("B\t",B)
    #print("Gendreau finish")
    return assign

## アイテム集合に与える順序、大きさが大きい順orコンフリクトの数が多い順

In [25]:
#入力, 出力を書き出す階層を指定
import os
path = os.path.abspath(".")
print(path)

/Users/okamoto/Desktop/yutaokamoto_mac/Seminar/benchmark


In [36]:
for instance_name in ["w120","w250","w500","w1000"]:
    for instance_number in range(10):
        for density in range(10):
                    
            if instance_number==0:# and density==1 and instance_name=="w250" :################################
                if instance_name=="w250" and density==1:

                    file_input = path + "/" + "input" + "/" + str("i"+str(instance_name[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + ".txt" #インスタンスの規模_インスタンスの番号_グラフの密度

                    n = None
                    V = None
                    C = 150
                    w = []
                    E = []
                    M = []
                    group_split_cost = 1
                    r = None
                    bin_cost = 5

                    with open(file_input,mode="r") as f:
                        for line in f:
                            if "Number of items\t" in line:
                                n = int(line.split("\t")[-2])
                                V = np.arange(n)
                            elif "Capacity of bins\t" in line:
                                C = int(line.split("\t")[-2])
                            elif "w\t" in line:
                                w.append(int(line.split("\t")[-1]))
                            elif "e\t" in line:
                                E.append(tuple(map(int,line.split("\t")[-1].rstrip("\n").strip("(").strip(")").split(","))))
                            elif "g\t" in line:
                                M.append(tuple(map(int,line.split("\t")[-1].rstrip("\n").strip("(").strip(")").strip(",").split(","))))
                            elif "Costs of spliting groups\t" in line:
                                r = [int(i) for i in line.split("\t")[-2][1:-1].split(",")]
                            elif "Costs of using bins\t" in line:
                                bin_cost = int(line.split("\t")[-2])


                    #アイテムの分割
                    ##コンフリクトの数が大きい順、大きさが大きい順にアイテムを並べ替える
                    E_dict = {}
                    E_dict = E_dict_func(E)
                    I = sorted(V,key=lambda x:(degree(x,E_dict), w[x]),reverse=True)
                    ##I1とI2の要素の個数を決める
                    n1 = None
                    n2 = None
                    n1 = int(np.ceil(n*bin_cost/(bin_cost+r[0])))
                    n2 = n-n1
                    #print("n1\t",n1)
                    #print("n2\t",n2)
                    #print()
                    ##I1とI2を定める
                    I1 = None
                    I2 = None
                    I1 = I[:n1]
                    I2 = I[n1:]
                    #print("I1\t",I1)
                    #print("I2\t",I2)
                    #print()
                    ##枝集合の更新
                    I1_E = None
                    I2_E = None
                    I1_E = update_E(E,I1)
                    I2_E = update_E(E,I2)
                    #print("I1_E\t",I1_E)
                    #print("I2_E\t",I2_E)
                    #print()

                    for heuristics in range(3,8):
                        if heuristics==3:

                            #ヒューリスティクスの選択
                            assign = {}
                            assign = Gendreau(I1,C,w,I1_E,M,assign)
                            #print(assign,"\n")
                            #item_set=set()
                            #double=set()
                            #for item in assign.values():
                                #double = (item_set&set(item)) | double
                                #item_set = item_set | set(item)
                            #if double!=set():
                                #print("アイテムの個数は",len(item_set),"\n")
                                #print("重複しているアイテムは",double,"\n")

                            #B = {}
                            #for b in assign.keys():
                                #B[b] = C-sum(w[i] for i in assign[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
                            #for b in B:
                                #if B[b]<0:
                                    #print("assign\t",assign)
                                    #print("B\t",B)
                                    #print()"""

                            #print("H"+str(heuristics)+"\n")

                            assign = eval("H"+str(heuristics))(I2,C,w,I2_E,M,r,assign)

                            #print(assign,"\n")
                            #item_set=set()
                            #double=set()
                            #for item in assign.values():
                                #double = (item_set&set(item)) | double
                                #item_set = item_set | set(item)
                            #if double!=set():
                                #print("アイテムの個数は",len(item_set),"\n")
                                #print("重複しているアイテムは",double,"\n\n\n")

                            #B = {}
                            #for b in assign.keys():
                                #B[b] = C-sum(w[i] for i in assign[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
                            #for b in B:
                                #if B[b]<0:
                                    #print("assign\t",assign)
                                    #print("B\t",B)
                                    #print("\n"*2)"""

                            #結果の出力　→　出力をテキストファイルに書き出す
                            bin_counter=len(assign.keys())

                            penalty_sum = 0
                            penalty_dict = {}
                            penalty_list = []
                            for items in assign.values():
                                penalty_list = list(set(group(item,M) for item in items)) #そのビンを使用しているグループの番号
                                for group_name in penalty_list:
                                    penalty_dict[group_name] = penalty_dict.setdefault(group_name,0) + 1
                            penalty_sum = sum(v*r[i] for i,v in penalty_dict.items())

                            #print("bin_counter\t",bin_counter)
                            #print("penalty_dict\t",penalty_dict)
                            #print("penalty_sum\t",penalty_sum)
                            #print("\n\n\n")


                            file_output =  path + "/" + "output" + "/" + "Myheuristics_Numba" + "/" + str("i"+str(instance_name[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + "_" + str("H"+str(heuristics)) + ".txt" #インスタンスの規模_インスタンスの番号_グラフの密度
                            with open(file_output,mode="w") as f:
                                f.write("Number of Bins\t"+str(bin_counter)+"\n\n")
                                f.write("Bin cost\t"+str(bin_cost)+"\n\n")
                                f.write("Penalty\t"+str(penalty_sum)+"\n\n")
                                f.write("obj\t"+str(penalty_sum+bin_counter*bin_cost))

# 小さい問題例

In [74]:
n = 6
V = np.arange(n)
C = 9
B = {i:C for i in range(n)}
w = [3, 2, 5, 6, 1, 1]
E = [(0,2), (1,2)]
M = [(0,3), (1,4,5), (2,)]
r = [1]*len(M)
bin_cost = 100


#アイテムの分割

##大きさが大きい順にアイテムを並べ替える
I = sorted(V,key=lambda x:w[x],reverse=True)

##I1とI2の要素の個数を決める
n1 = None
n2 = None
n1 = 3#int(np.ceil(n*bin_cost/(bin_cost+r[0])))
n2 = n-n1
print("n1\t",n1)
print("n2\t",n2)
print()

##I1とI2を定める
I1 = None
I2 = None
I1 = I[:n1]
I2 = I[n1:]
print("I1\t",I1)
print("I2\t",I2)
print()

##枝集合の更新
I1_E = None
I2_E = None
I1_E = update_E(E,I)
I2_E = update_E(E,I2)
print("I1_E\t",I1_E)
print("I2_E\t",I2_E)
print()


#ヒューリスティクスの選択
heuristics = 7
print("heuristics\t",heuristics)
group_order = 1
print("group_order\t",group_order)
print()

assign = {}
assign = Gendreau(I1,C,w,I1_E,M,assign)
B = {}
for b in assign.keys():
    B[b] = C-sum(w[i] for i in assign[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
print("assign\t",assign)
print("B\t",B)
print()

assign = eval("H"+str(heuristics))(I2,C,w,I2_E,M,r,group_order,assign)
B = {}
for b in assign.keys():
    B[b] = C-sum(w[i] for i in assign[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
print("assign\t",assign)
print("B\t",B)
print()

#結果の出力　→　出力をテキストファイルに書き出す
bin_counter=len(assign.keys())

penalty_sum = 0
penalty_dict = {}
penalty_list = []
for items in assign.values():
    penalty_list = list(set(group(item,M) for item in items)) #そのビンを使用しているグループ名
    for group_name in penalty_list:
        penalty_dict[group_name] = penalty_dict.setdefault(group_name,0) + 1
        penalty_sum += r[group_name]*(penalty_dict[group_name]-1)
        
print("bin_counter\t",bin_counter)
print("penalty_sum\t",penalty_sum)

n1	 3
n2	 3

I1	 [3, 2, 0]
I2	 [1, 4, 5]

I1_E	 [(0, 2), (1, 2)]
I2_E	 [(1, 2)]

heuristics	 7
group_order	 1

assign	 {0: [0, 3], 1: [2]}
B	 {0: 0, 1: 4}

assign	 {0: [0, 3], 1: [2, 4, 5], 2: [1]}
B	 {0: 0, 1: 2, 2: 7}

bin_counter	 3
penalty_sum	 1


In [73]:
M

[(0, 3), (1, 4, 5), (2,)]

In [69]:
group_order

1